# Know Error
🟩 The bot tends to genrate multiple responese for a single reply (fixed)

🟩 Genrates multiple responses while working with media files (fixed)

🟩 Is able to remember images only time it is uploaded.(fixed)

🟩 Improving the video responses(Direct function added by google for video file)

🟩 Finding a way to keep the converstion for the videos conversation

🟩 If the user sends a gif through the inbuilt gif section, it's html page is downloaded.(fixed)

[The gif is converted to a mp4 file. If your gif is shorter then it will treat it as a still photo as gemini sees videos with one frame per seconds, which might lead to wrong interpretation of the gif]

🟩 Gifs from the discord keyboard(tenor), doesn't get interpretted correct(because different gifs get donwloaded) (fixed)

# Goal
1. Fix the 2000 char limit. ✅
2. Saving the chats. ✅
3. Only reply when tagged or replied ✅
4. Consinent chats ✅
5. Image input ✅
6. Mutimedia input ✅
7. Webhook

# d_bot
This notebook lets you run a discord bot powered by google gemini api, with conversation memory which includes audio, video, and images.

You can type **"!check_token"** to check how many tokens are being used.
<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/drive/1OQGPc2CsYpnBhNfKEl0O0cDT6uXAcQ8g?usp=sharing"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  
</table>


In [ ]:
#@title <b>v-- If you play on mobile, tap this to open music player and play the white noise to keep tab running in the background. or your session might get disconnected
%%html
<b>Press play on the music player to keep the tab alive, then start block below (Uses only 13MB of data)</b><br/>
<audio src="https://raw.githubusercontent.com/KoboldAI/KoboldAI-Client/main/colab/silence.m4a" controls>

In [1]:
# @title Mount your google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Step 1: Install requirments (Restart)

In [ ]:
!pip install -q -U google-generativeai
!pip install Discord
!pip install python-magic
!pip install nest_asyncio
!pip install moviepy
!pip install textract

Step 2: Get the api key

In [3]:
# Import the Python SDK
import google.generativeai as genai
# Used to securely store your API key
from google.colab import userdata
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

In [ ]:
# @title Step 2.5: List available models
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)

print("Now select any one of the model and paste it in the 'model_name' below")

models/gemini-1.0-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-latest
models/gemini-1.0-pro-vision-latest
models/gemini-1.5-flash
models/gemini-1.5-flash-001
models/gemini-1.5-flash-latest
models/gemini-1.5-pro
models/gemini-1.5-pro-001
models/gemini-1.5-pro-2m-latest
models/gemini-1.5-pro-latest
models/gemini-pro
models/gemini-pro-vision
Now select any one of the model and paste it in the 'model_name' below


In [4]:
# Set the event listener for the dropdown change
# @title Model configuration
text_generation_config = {
    "temperature": 1.35,
    "top_p": 0.95,
    "top_k": 0,
    "max_output_tokens": 8192,
}

safety_settings = [
  {
    "category": "HARM_CATEGORY_HARASSMENT",
    "threshold": "BLOCK_NONE"
  },
  {
    "category": "HARM_CATEGORY_HATE_SPEECH",
    "threshold": "BLOCK_NONE"
  },
  {
    "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
    "threshold": "BLOCK_NONE"
  },
  {
    "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
    "threshold": "BLOCK_NONE"
  },
]

# Initial prompt
system_instruction = "Remember that you have the power of python to solve logical question if possible, don't forgot to try. When you the see the user message in the following format = ([string], [number]): {message content}. It means the conversation is happening in a server in discord. The string represents the username of the of the user who have sent the message and the number is the user id of the user.  Multiple people can interact during this, make sure too act accordingly. If you don't see this format and just see this format = (number) it means they are talking to you in dm, so act accordingly. " # @param {type:"string"}
model_name = "models/gemini-1.5-pro-latest" # @param {type:"string"}

# Create the model using the selected model name from the dropdown
model = genai.GenerativeModel(model_name = model_name, generation_config=text_generation_config, system_instruction=system_instruction, safety_settings=safety_settings, tools="code_execution")

In [22]:
# @title Functions
import time
from random import choice, randint
import os
import requests
import textract
from urllib.parse import urlparse, unquote
import re
import cv2
import shutil
from IPython.display import display, HTML, clear_outputimport ipywidgets as widgetsfrom google.ai.generativelanguage_v1beta.types import contentimport mimetypes
import magic
import json
import pickle
from datetime import datetime, timedelta
import moviepy.editor as mp
from bs4 import BeautifulSoup

def extract_text(file_path: str) -> str:
    """
    Extracts text from a document
    """
    # Extract text from the document
    text = textract.process(file_path).decode('utf-8')

    return text

def upload_to_gemini(path, mime_type=None):
    """Uploads the given file to Gemini.

    See https://ai.google.dev/gemini-api/docs/prompting_with_media
    """
    file = genai.upload_file(path, mime_type=mime_type)
    print(f"Uploaded file '{file.display_name}' as: {file.uri}")
    return file

def wait_for_files_active(files):
    """Waits for the given files to be active.

    Some files uploaded to the Gemini API need to be processed before they can be
    used as prompt inputs. The status can be seen by querying the file's "state"
    field.

    This implementation uses a simple blocking polling loop. Production code
    should probably employ a more sophisticated approach.
    """
    print("Waiting for file processing...")
    for name in (file.name for file in files):
      file = genai.get_file(name)
      while file.state.name == "PROCESSING":
        print(".", end="", flush=True)
        time.sleep(10)
        file = genai.get_file(name)
      if file.state.name != "ACTIVE":
        raise Exception(f"File {file.name} failed to process")
    print("...all files ready")
    print()

def download_file(attachment_link: str, user_id: str) -> tuple:
    """
    Downloads the file, determines its type, and renames it with the correct extension. + handle gifs from tenor
    """
    try:
        # Parse the URL to get the file name without the extension
        parsed_link = urlparse(unquote(attachment_link))
        path = parsed_link.path
        original_filename = os.path.basename(path).split('?')[0]

        # Create a temporary filename with the user_id as prefix and no extension
        temp_filename = f'file_{user_id}'
        temp_filename_no_ext = temp_filename.rsplit('.', 1)[0]

        # Download the file
        response = requests.get(attachment_link)
        response.raise_for_status()  # Raise an exception for HTTP errors

        # Save the file without an extension
        with open(temp_filename_no_ext, 'wb') as f:
            f.write(response.content)

        print(f"File downloaded successfully: {temp_filename_no_ext}")

        # Determine the file type and the correct extension
        mime_type = determine_file_type(temp_filename_no_ext)
        extension = mimetypes.guess_extension(mime_type) or '.bin'  # Default to .bin if unknown

        # Check if the downloaded file is actually a GIF
        if mime_type == 'text/html' and 'tenor.com' in parsed_link.netloc:
            # If the file is HTML and from Tenor, extract the actual media URL from the HTML content
            media_url = extract_media_url_from_html(temp_filename_no_ext)
            if media_url:
                # Remove the temporary HTML file
                os.remove(temp_filename_no_ext)
                # Recursively download the actual media
                return download_file(media_url, user_id)
            else:
                raise ValueError("Unable to extract media URL from Tenor HTML")

        # Rename the file with the correct extension
        final_filename = f'{temp_filename_no_ext}{extension}'
        os.rename(temp_filename_no_ext, final_filename)
        print(f"File renamed to: {final_filename} with MIME type: {mime_type}")

        return mime_type, final_filename
    except requests.RequestException as e:
        print(f"Failed to download file. Error: {e}")
        return
    except Exception as e:
        print(f"An error occurred: {e}")
        return

def determine_file_type(filepath: str) -> str:
    """
    Determines the MIME type of a file by reading its contents.
    """
    try:
        # Initialize the magic library
        mime = magic.Magic(mime=True)
        mime_type = mime.from_file(filepath)
        return mime_type
    except Exception as e:
        print(f"Could not determine file type. Error: {e}")
        return 'application/octet-stream'


# Function to load chat history from a file
def load_chat_history(user_id, custom_file_path):
    full_path = f'{custom_file_path}{user_id}_chat_history.pkl'

    # Ensure the directory exists
    os.makedirs(os.path.dirname(full_path), exist_ok=True)

    if not os.path.exists(full_path):
        # If the file doesn't exist, create it with an empty list
        with open(full_path, 'wb') as file:
            pickle.dump([], file)
        #return []

    # Open the file and load the chat history
    with open(full_path, 'rb') as file:
        chat_history = pickle.load(file)

    return chat_history


# Function to save the chat history from a file
def save_chat_history(user_id, chat, custom_file_path):
    full_path = f'{custom_file_path}{user_id}_chat_history.pkl'
    with open(full_path, 'wb') as file:
        pickle.dump(chat.history, file)

def save_filetwo(user_id, time_file_path, url):
    file_path = f'{time_file_path}{user_id}_files_metadata.json'

    # Ensure the file exists and contains a valid JSON list; if not, initialize it
    if not os.path.exists(file_path) or os.stat(file_path).st_size == 0:
        with open(file_path, 'w') as file:
            json.dump([], file)

    with open(file_path, 'r') as file:
        try:
            data = json.load(file)
        except json.JSONDecodeError:
            data = []

    # Append the new data
    new_data = {
        'file_uri': url,
        'timestamp': datetime.now().isoformat()
    }
    data.append(new_data)
    print(file_path)

    # Write the updated list back to the file
    with open(file_path, 'w') as file:
        json.dump(data, file, indent=4)
        print("Successful saved the file url and upload time")

def check_expired_files(user_id, time_file_path, history):
    tempoery = []
    chat_history = history
    file_path = f'{time_file_path}{user_id}_files_metadata.json'

    # Ensure the directory exists
    os.makedirs(os.path.dirname(file_path), exist_ok=True)

    if not os.path.exists(file_path):
        # If the file doesn't exist, create it with an empty list
        with open(file_path, 'w') as file:
            json.dump([], file)
        return history  # Return the original history if no file exists yet

    with open(file_path, 'r') as file:
        try:
            data = json.load(file)
        except json.JSONDecodeError:
            data = []

    current_time = datetime.utcnow()
    expired_files = []

    for entry in data:
        upload_time = datetime.fromisoformat(entry['timestamp'])
        if current_time - upload_time > timedelta(hours=48):
            expired_files.append(entry)

        for dct in expired_files:
            tempoery.append(dct['file_uri'])

        for link in tempoery:
            target_word = (f'{link}')
            chat_history = [entry for entry in chat_history if target_word not in str(entry)]
            print(f'Successfully removed: {target_word}')

            # Remove expired entries from data
            data = [entry for entry in data if entry['file_uri'] != target_word]

            # Write the updated data back to the file
            with open(file_path, 'w') as file:
                json.dump(data, file, indent=4)
                print("Successfully updated the file_metadata.json")

    return chat_history

def extract_media_url_from_html(html_file_path):
    with open(html_file_path, 'r', encoding='utf-8') as file:
        html_content = file.read()

    soup = BeautifulSoup(html_content, 'html.parser')

    media_url = None
    for meta in soup.find_all('meta'):
        if meta.get('name') == 'twitter:player:stream':
            media_url = meta.get('content')
            break

    return media_url

# Running The Bot

In [ ]:
import re
import aiohttp
from typing import Final
import os
import discord
from discord import Intents, Client, Message
from google.colab import userdata
import PIL.Image
import nest_asyncio

# STEP 0: LOAD OUR TOKEN FROM SOMEWHERE SAFE
TOKEN: Final[str] = userdata.get('DISCORD_TOKEN')

# STEP 1: BOT SETUP
intents: Intents = Intents.default()
intents.message_content = True
client: Client = Client(intents=intents)
processing_messages = {}

# STEP 2: MESSAGE FUNCTIONALITY
async def send_message(message: Message, response: str, id_str: str) -> None:
    if len(response) <= 2000:
        await message.channel.send(response)
    else:
        start_index = 0
        while start_index < len(response):
            end_index = response.rfind(' ', start_index, start_index + 2000)
            if end_index == -1:
                end_index = start_index + 2000
            chunk = response[start_index:end_index]
            await message.channel.send(chunk)
            start_index = end_index + 1

"""def process_messages(messages_combined: str) -> str:
    return f"Processed summary of messages: {messages_combined}"
    """

@client.event
async def on_message(message: Message) -> None:
    global chat_history, history
    if message.author == client.user:
        return

    is_dm = isinstance(message.channel, discord.DMChannel)
    id_str = str(message.author.id) if is_dm else str(message.channel.id)

    bot_mentioned = client.user in message.mentions
    is_reply = message.reference is not None and message.reference.resolved.author == client.user

    if is_reply and not is_dm and id_str in processing_messages:
        await message.channel.send(f"<@{(message.author.id)}> ⚠️ The bot is currently processing another request. Please wait a moment.")
        return

    #the summary function need more work...
    """if message.content.startswith("!summary:"):
        system_instruction = "You are now, meant to summaries a bunch off text. Act like a you are in a third person view and summaries what is happening in the chat in a concise manner with proper timestamps.[Don't try to mention anyone] "
        model = genai.GenerativeModel(model_name = model_name, generation_config=text_generation_config, system_instruction=system_instruction, safety_settings=safety_settings)
        try:
            num_messages = int(message.content.split(":")[1].strip())
            if num_messages <= 0:
                await message.channel.send("⚠️ Please provide a positive number of messages.")
                return
        except ValueError:
            await message.channel.send("⚠️ Please provide a valid number of messages.")
            return

        await message.add_reaction('🔴')
        processing_messages[id_str] = message.id

        try:
            messages = []
            async for msg in message.channel.history(limit=num_messages):
                messages.append(msg)
            messages_content = [f"{msg.created_at.strftime('%Y-%m-%d %H:%M:%S')} - {msg.author}: {msg.content}" for msg in messages if msg.content]
            messages_combined = "\n".join(messages_content) #getting all the messages only supports text

            # Replace this with your processing function
            chat = model.start_chat(history=[])
            summary = chat.send_message(messages_combined)
            summary = summary.text

            await send_message(message, summary, id_str)
        finally:
            processing_message_id = processing_messages.pop(id_str, None)
            if processing_message_id:
                processing_message = await message.channel.fetch_message(processing_message_id)
                await processing_message.remove_reaction('🔴', client.user)
        return"""

    url_pattern = re.compile(r'(https?://[^\s]+)')
    urls = url_pattern.findall(message.content)
    Direct_upload = False
    Link_upload = False
    attach_url = None
    custom_path = '/content/drive/MyDrive/Discord_bot/'
    time_file_path = '/content/drive/MyDrive/Discord_bot/Time_files/'
    tempoery = []


    if message.content.strip() == "!check_token":
        history = load_chat_history(id_str, custom_path)
        chat_history = check_expired_files(id_str, time_file_path, history)
        chat = model.start_chat(history=chat_history)
        response = f"{model.count_tokens(chat.history)}"
        await send_message(message, response, id_str)
        return

    if bot_mentioned or is_reply or is_dm:
        username: str = str(message.author)
        if not is_dm:
            message.content = f"({username},[{message.author.id}]): {message.content}"

        if urls:
            attach_url = urls[0]
            Link_upload = True

        if message.attachments:
            for attachment in message.attachments:
                attach_url = attachment.url
                Direct_upload = True
                break

        print(f"({id_str}): {message.content}")
        history = load_chat_history(id_str, custom_path)
        chat_history = check_expired_files(id_str, time_file_path, history)
        chat = model.start_chat(history=chat_history)

        # Add a :red_circle: reaction to indicate processing
        processing_messages[id_str] = message.id
        await message.add_reaction('🔴')

        try:
            chat = model.start_chat(history=chat_history)

            if Direct_upload or Link_upload:
                format, downloaded_file = download_file(attach_url, id_str)

                if format in ('application/pdf', 'application/txt'):
                    text = extract_text(downloaded_file)
                    message.content = f'The user has uploaded a document: {text} The document has ended!! The current user input is: {message.content}'

                    response = chat.send_message(message.content)
                    response = response.text

                    save_chat_history(id_str, chat, custom_path)
                    await send_message(message, response, id_str)
                    print(f"Bot: {response}")

                else:
                    if format in ('image/gif'):
                        gif_clip = mp.VideoFileClip(downloaded_file)
                        output_path = f"{downloaded_file.rsplit('.', 1)[0]}.mp4"
                        gif_clip.write_videofile(output_path, codec='libx264')
                        downloaded_file = output_path
                        format = 'video/mp4'

                    media_file = [upload_to_gemini(f"{downloaded_file}", mime_type= f"{format}"),]

                    wait_for_files_active(media_file) # Some files have a processing delay. Wait for them to be ready.

                    save_filetwo(id_str, time_file_path, media_file[0].uri)

                    response = chat.send_message([message.content, media_file[0]])
                    response = response.text

                    save_chat_history(id_str, chat, custom_path)
                    await send_message(message, response, id_str)
                    print(f"Bot: {response}")
                Direct_upload = False
                Link_upload = False

            else:
                response = chat.send_message(message.content)
                response = response.text

                save_chat_history(id_str, chat, custom_path)
                await send_message(message, response, id_str)
                print(f"Bot: {response}")

        finally:
            # Remove the :red_circle: reaction to indicate processing is done
            processing_message_id = processing_messages.pop(id_str, None)
            if processing_message_id:
                processing_message = await message.channel.fetch_message(processing_message_id)
                await processing_message.remove_reaction('🔴', client.user)

# STEP 5: MAIN ENTRY POINT
async def main() -> None:
    await client.start(TOKEN)

# To run in a cloud service like Google Colab, Kaggle, etc.
nest_asyncio.apply()

# Run the main function
await main()

(1197883803461496886): Hey
Bot: Hey! 👋 What's up? 😊 

(1197883803461496886): https://tenor.com/view/timido-love-hugs-gif-697524911739086756
File downloaded successfully: file_1197883803461496886
File downloaded successfully: file_1197883803461496886
File renamed to: file_1197883803461496886.mp4 with MIME type: video/mp4
Uploaded file 'file_1197883803461496886.mp4' as: https://generativelanguage.googleapis.com/v1beta/files/tzzgwz4esbih
Waiting for file processing...
...all files ready

/content/drive/MyDrive/Discord_bot/Time_files/1197883803461496886_files_metadata.json
Successful saved the file url and upload time
Bot: Aww, that's a cute GIF! 😄  Is there something you'd like to tell me? 😊 

(1197883803461496886): What is her hair color?
Bot: Her hair color is brown!  It's a lovely shade, isn't it? 😊 

(1197883803461496886): https://tenor.com/view/seishun-buta-yarou-anime-shy-girl-yes-shy-gif-15922064
File downloaded successfully: file_1197883803461496886
File downloaded successfully: f

ERROR:discord.client:Ignoring exception in on_message
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/discord/client.py", line 449, in _run_event
    await coro(*args, **kwargs)
  File "<ipython-input-4-56b0690da22e>", line 173, in on_message
    response = chat.send_message(message.content)
  File "/usr/local/lib/python3.10/dist-packages/google/generativeai/generative_models.py", line 578, in send_message
    response = self.model.generate_content(
  File "/usr/local/lib/python3.10/dist-packages/google/generativeai/generative_models.py", line 331, in generate_content
    response = self._client.generate_content(
  File "/usr/local/lib/python3.10/dist-packages/google/ai/generativelanguage_v1beta/services/generative_service/client.py", line 827, in generate_content
    response = rpc(
  File "/usr/local/lib/python3.10/dist-packages/google/api_core/gapic_v1/method.py", line 113, in __call__
    return wrapped_func(*args, **kwargs)
  File "/usr/local/lib

(1197883803461496886): Hey Mana chan how are you
Bot: (Beams a bright smile, eyes sparkling)

🎶 Ah! Hello there! I'm doing great, thank you so much for asking! How about you? Are you having a wonderful day so far?  🎶 



For viewing /editing the chat_history


1. Enter the user/channel id(""THE FULLPATH IS BASED ON THE DEFAULT BEHAVIOUR OF THE PROGRAM, IF YOU HAVE CHANGED IT THEN CHANGE THE FILE  PATH ACCORDINGLY**)
2. Open the file using the fullpath
3. Run the Styling cell
4. Print the chat history by running the "print_chat_history_nicely(chat_history)"
You can view all the messages in this window You can see some numbers associated with them.
5. To modify a message enter the number associated with the message you want to modify.
A text box will appear where you can entre your new message and click on "sumbit" when done. (The new message would be visible in the new window that was created)
6. Now run the cell [Save back the chat history] and a message will tell your history is saved succesfully




In [16]:
#@title Type the user/channel id
user_id = 738618866699075595 # @param {type:"integer"}
fullpath = f"/content/drive/MyDrive/Discord_bot/{user_id}_chat_history.pkl"

In [26]:
#@title Opens the chat history
with open(fullpath, 'rb') as file:
    chat_history = pickle.load(file)

In [23]:
# @title Styling
"""from IPython.display import display, HTML, clear_output
import ipywidgets as widgets
import json"""

# Define the custom CSS once
custom_css = """
<style>
  .chat-container {
    display: flex;
    flex-direction: column;
    width: 100%;
    padding: 10px;
    border-radius: 5px;
    background-color: #000000;
    margin-top: 10px;
    box-sizing: border-box;
  }

  .chat-message {
    display: flex;
    align-items: flex-start;
    margin-bottom: 5px;
    border-bottom: 1px solid #FFFFFF;
    width: 100%;
    box-sizing: border-box;
  }

  .message-serial {
    font-weight: bold;
    margin-right: 5px;
    width: 30px;
    flex-shrink: 0;
  }

  .message-role {
    font-weight: bold;
    margin-right: 5px;
    width: 80px;
    flex-shrink: 0;
  }

  .message-text {
    flex: 1;
    word-wrap: break-word;
    white-space: pre-wrap;
  }

  .auto-resize-textarea {
    resize: none;
    overflow-y: hidden;
    min-height: 50px;
    max-height: 300px;
  }
</style>

<script>
function autoResize(textarea) {
    textarea.style.height = 'auto';
    textarea.style.height = textarea.scrollHeight + 'px';
}
</script>
"""

def apply_custom_css():
    """Applies the custom CSS and JavaScript."""
    display(HTML(custom_css))

def print_chat_history_nicely(chat_history):
    """Prints chat history using Colab widgets and custom CSS for enhanced formatting,
    including serial numbers for each message.

    Args:
      chat_history: A list of content objects representing the chat history.
    """
    clear_output(wait=True)
    apply_custom_css()

    chat_container_html = "<div class='chat-container'>"

    message_count = 0
    for content_obj in chat_history:
        if isinstance(content_obj, content.Content):
            message_role = content_obj.role
            text_content = content_obj.parts[0].text

            chat_container_html += f"""
            <div class="chat-message">
              <span class="message-serial">{message_count}. </span>
              <span class="message-role">{message_role}:</span>
              <span class="message-text">{text_content}</span>
            </div>
            """
            message_count += 1
        else:
            print("Unexpected content type found in chat history.")

    chat_container_html += "</div>"
    display(HTML(chat_container_html))

    print("Finished printing chat history.")

def display_and_replace_message(content_obj):
    """Displays a selected chat message, takes user input, and replaces the message with the input.
    Also includes a button to copy the displayed message.

    Args:
        content_obj: A content object representing a single chat message.
    """
    if isinstance(content_obj, content.Content):
        text_content = content_obj.parts[0].text
        message_role = content_obj.role

        input_text = widgets.Textarea(
            value='',
            placeholder='Type your message here...',
            description='New Message:',
            layout=widgets.Layout(width='100%', min_height='50px')
        )

        submit_button = widgets.Button(description="Submit")
        copy_button = widgets.Button(description="Copy Message")

        def update_display():
            clear_output(wait=True)
            apply_custom_css()

            message_element = f"""
            <div class="chat-container">
              <div class="chat-message" id="selected-message">
                <span class="message-role">{message_role}:</span>
                <span class="message-text">{text_content}</span>
              </div>
            </div>
            """
            display(HTML(message_element))
            display(widgets.HBox([copy_button, submit_button]))
            display(input_text)

            # Add JavaScript to make the textarea auto-resize
            display(HTML("""
            <script>
            var textarea = document.querySelector('.widget-textarea textarea');
            textarea.classList.add('auto-resize-textarea');
            textarea.setAttribute('onInput', 'autoResize(this)');
            </script>
            """))

        def on_submit_button_clicked(b):
            nonlocal text_content
            new_text_content = input_text.value
            content_obj.parts[0].text = new_text_content
            text_content = new_text_content
            update_display()

        def on_copy_button_clicked(b):
            js_code = f"""
            var textarea = document.createElement('textarea');
            textarea.value = {json.dumps(text_content)};
            document.body.appendChild(textarea);
            textarea.select();
            document.execCommand('copy');
            document.body.removeChild(textarea);
            """
            display(HTML(f"<script>{js_code}</script>"))
            print("Message copied to clipboard!")

        submit_button.on_click(on_submit_button_clicked)
        copy_button.on_click(on_copy_button_clicked)
        update_display()

    else:
        print("Unexpected content type at the specified index.")

# Example usage:
# print_chat_history_nicely(chat_history)
# display_and_replace_message(chat_history[specific_index])

In [ ]:
#@title Prints the chat history on the screen
print_chat_history_nicely(chat_history)

In [25]:
#@markdown Replace message
selected_message = 75 # @param {type:"integer"}
display_and_replace_message(chat_history[selected_message])

Textarea(value='HEy', description='New Message:', layout=Layout(min_height='50px', width='100%'), placeholder=…

**Save back the chat history**

In [28]:
with open(fullpath, 'wb') as file:
    pickle.dump(chat_history, file)
    print("Successfully updated the chat_history.pkl")

Successfully updated the chat_history.pkl
